## Importing module

In [1]:
import pandas as pd 
import numpy as np
import seaborn as sns
from ipywidgets import interact
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
import itertools
  

## Importing data

In [2]:
df=pd.read_csv("Oil and Gas 1932-2014.csv")
df.shape

(15521, 41)

## Adding the global Oil and Gas production

In [3]:
df['global_oil_prod']=list(df[['year','oil_prod32_14']].groupby(['year']).sum().reset_index().iloc[:,1])*len(df['cty_name'].unique())
df['global_gas_prod']=list(df[['year','gas_prod55_14']].groupby(['year']).sum().reset_index().iloc[:,1])*len(df['cty_name'].unique())

## Checking For Duplicates values

In [4]:
df.drop_duplicates().shape

(15521, 43)

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15521 entries, 0 to 15520
Data columns (total 43 columns):
cty_name                        15521 non-null object
iso3numeric                     15521 non-null int64
id                              15106 non-null object
year                            15521 non-null int64
eiacty                          14254 non-null object
oil_prod32_14                   10103 non-null float64
oil_price_2000                  15503 non-null float64
oil_price_nom                   15521 non-null float64
oil_value_nom                   10103 non-null float64
oil_value_2000                  10103 non-null float64
oil_value_2014                  10103 non-null float64
gas_prod55_14                   8812 non-null float64
gas_price_2000_mboe             15521 non-null float64
gas_price_2000                  11220 non-null float64
gas_price_nom                   15521 non-null float64
gas_value_nom                   8812 non-null float64
gas_value_2000      

## Converting sovereign into Categorical

In [6]:
df[['sovereign']] = df[['sovereign']].astype('category')
df.sovereign.dtype

CategoricalDtype(categories=[0, 1], ordered=False)

# Checking the Null Ratio

In [7]:
n = len(df)
l=pd.DataFrame(df.isnull().sum()).reset_index()
l.columns = ['Colname','ratio_null']
l['ratio_null']= l.ratio_null/n
print(l.sort_values('ratio_null',ascending=False))

                         Colname  ratio_null
34  net_oil_gas_exports_valuePOP    0.750338
33      net_gas_exports_valuePOP    0.750338
32         net_gas_exports_value    0.750016
31          net_gas_exports_mboe    0.741254
30           net_gas_exports_bcf    0.741254
29                   gas_exports    0.741125
28      net_oil_exports_valuePOP    0.709748
24                   oil_exports    0.709426
25               net_oil_exports    0.709426
26            net_oil_exports_mt    0.709426
27         net_oil_exports_value    0.709426
11                 gas_prod55_14    0.432253
15                 gas_value_nom    0.432253
17                gas_value_2014    0.432253
16                gas_value_2000    0.432253
21          oil_gas_valuePOP_nom    0.351073
22         oil_gas_valuePOP_2000    0.351073
23         oil_gas_valuePOP_2014    0.351073
10                oil_value_2014    0.349075
9                 oil_value_2000    0.349075
8                  oil_value_nom    0.349075
5         

In [8]:
df.mult_2000_2014.unique()
df.replace(np.nan,0,inplace=True)

# Correlation Techniques

## Segregating Data

In [9]:
oil_col=[i for i  in df.columns if 'oil' in i and 'gas' not in i and 'export' not in i and 'value_2014' not in i ];oil_col.extend(['oil_value_2014','gas_value_2014'])
gas_col=[i for i  in df.columns if 'gas' in i and 'oil' not in i and 'export' not in i];gas_col.extend(['oil_value_2014',])
oil_gas_col=[i for i  in df.columns if 'gas' in i and 'oil'  in i and 'export' not in i];oil_gas_col.extend(['oil_value_2014','gas_value_2014'])
export_col=[i for i  in df.columns if  'export'  in i];export_col.extend(["oil_value_2014",'gas_value_2014'])
other_col= ['year','population','sovereign','mult_nom_2000','mult_nom_2014','oil_value_2014','gas_value_2014']

## Correlation Matrix

In [10]:
def heatmap(data): 
    fig = plt.figure(figsize=(10,10));ax1 = fig.add_subplot();grid = round(df[eval(data)].replace(0,np.nan).dropna().corr(),2);plt.imshow(grid,cmap=plt.cm.get_cmap('coolwarm'));plt.colorbar()
    c = [""];c.extend(grid.columns);ax1.set_xticks(np.arange(-1,grid.shape[1]+0.5));ax1.set_yticks(np.arange(-1,grid.shape[1]+0.5))
    ax1.set_xticklabels(c,rotation='vertical');ax1.set_yticklabels(c)
    for (j,i),label in np.ndenumerate(grid):
        ax1.text(i,j,label,ha='center',va='center')         
interact(heatmap,data=['oil_col','gas_col','oil_gas_col','export_col','other_col'])    

interactive(children=(Dropdown(description='data', options=('oil_col', 'gas_col', 'oil_gas_col', 'export_col',…

<function __main__.heatmap(data)>

## Top  Features  Correlated With Target

In [11]:
def correlation(Target,cty):
        v=pd.DataFrame(df.iloc[:,5:40].corr().loc[:,Target].reset_index())
        cor_table=v.loc[(v[Target]>0.3) | (v[Target]<-0.1) ][['index',Target]].sort_values(Target,ascending=False);print('______For Whole Data________');print(cor_table)
        print(f"Total Feature that we are taking into consideration is {len(cor_table)} Feature")
        print('__________________________________________City Wise Correlation Matrix______________')
        fig = plt.figure(figsize=(12,12))
        ax = fig.add_subplot();grid = round(df[df['cty_name']==cty][cor_table['index']].corr(),2);plt.imshow(grid,cmap=plt.cm.get_cmap('coolwarm'));plt.colorbar()
        c = [""];c.extend(grid.columns);ax.set_xticks(np.arange(-1,grid.shape[1]+0.5));ax.set_yticks(np.arange(-1,grid.shape[1]+0.5))
        ax.set_xticklabels(c,rotation='vertical');ax.set_yticklabels(c)
            
        for (j,i),label in np.ndenumerate(grid):
                ax.text(i,j,label,ha='center',va='center')
        
interact(correlation,Target=['oil_value_2014','gas_value_2014'],cty=sorted(set(df.iloc[:,0])))

interactive(children=(Dropdown(description='Target', options=('oil_value_2014', 'gas_value_2014'), value='oil_…

<function __main__.correlation(Target, cty)>

# Reverse Engineering

## Checking With R-square and significance

In [12]:
import statsmodels.formula.api  as smf

In [13]:
gas_coll="gas_prod55_14+gas_price_2000_mboe+gas_price_2000+gas_price_nom+gas_value_nom+gas_value_2000"
oil_coll="oil_prod32_14+oil_price_2000+oil_value_nom+oil_value_2000+oil_price_nom"
oil_gas_coll="oil_gas_value_nom+oil_gas_value_2000+oil_gas_value_2014+oil_gas_valuePOP_nom+oil_gas_valuePOP_2000+oil_gas_valuePOP_2014"
export_coll="oil_exports+net_oil_exports+net_oil_exports_mt+net_oil_exports_value+net_oil_exports_valuePOP+gas_exports+net_gas_exports_bcf+net_gas_exports_mboe+net_gas_exports_value+net_gas_exports_valuePOP+net_oil_gas_exports_valuePOP"
other_coll= "population+sovereign+mult_nom_2000+mult_nom_2014"
for_2000="oil_prod32_14+oil_price_2000+oil_value_nom+oil_price_nom+oil_value_2000+gas_prod55_14+gas_price_2000_mboe+gas_price_nom+gas_value_nom+gas_value_2000+oil_gas_value_nom+oil_gas_value_2000+oil_gas_valuePOP_nom+oil_gas_valuePOP_2000+sovereign+mult_nom_2000"
for_2014="oil_prod32_14+oil_price_2000+oil_value_nom+oil_price_nom+gas_prod55_14+gas_price_2000_mboe+gas_price_nom+gas_value_nom+gas_value_2014+oil_gas_value_nom+oil_gas_value_2014+oil_gas_valuePOP_nom+oil_gas_valuePOP_2014+sovereign+mult_nom_2014"
Final_oil="oil_value_2014~year+oil_price_nom+oil_gas_value_2014+oil_value_nom+gas_prod55_14+gas_value_2014+gas_exports"
Final_gas="gas_value_2014~year+gas_value_nom+gas_prod55_14+oil_gas_value_2014+oil_prod32_14+gas_exports+oil_value_2014+mult_nom_2014"
Single_value_oil="oil_value_2014~oil_value_2000"
Single_value_gas="gas_value_2014~gas_value_2000"
ls = ['Final_oil','Final_gas','Single_value_gas','Single_value_oil']
def test(line,Target):
    if line not in ls:
        T=[Target]
        T.append(eval(line))
        line="~".join(T)
        k=line.split("~")[1].split("+");k.extend(line.split("~")[0].split("+"))
        lr = smf.ols(line,data=df[k]).fit()
    else:
        k=eval(line).split("~")[1].split("+");k.extend(eval(line).split("~")[0].split("+"))
        lr = smf.ols(eval(line),data=df[k]).fit()
    print(lr.summary())
interact(test,Target=['oil_value_2014','gas_value_2014'],line=['oil_coll','gas_coll','oil_gas_coll','export_coll','other_coll','for_2000','for_2014','Final_oil','Final_gas','Single_value_gas','Single_value_oil'])

interactive(children=(Dropdown(description='line', options=('oil_coll', 'gas_coll', 'oil_gas_coll', 'export_co…

<function __main__.test(line, Target)>

## Checking with Loss Function(MAPE)

In [14]:
no = ['Armenia', 'Bahamas, The', 'Bhutan', 'Bosnia and Herzegovina', 'Botswana', 'Burkina Faso', 'Burundi', 'Cambodia', 'Cape Verde', 'Central African Republic', 'Comoros', 'Costa Rica', 'Cyprus', 'Djibouti', 'Dominican Republic', 'El Salvador', 'Eritrea', 'Ethiopia', 'Ethiopia including Eritrea', 'Fiji', 'Gambia, The', 'Guinea', 'Guinea-Bissau', 'Guyana', 'Haiti', 'Honduras', 'Iceland', 'Jamaica', 'Kenya', 'Korea, Dem. Rep.', 'Lao PDR', 'Latvia', 'Lebanon', 'Lesotho', 'Liberia', 'Macedonia, FYR', 'Madagascar', 'Malawi', 'Maldives', 'Mali', 'Malta', 'Mauritius', 'Montenegro', 'Namibia', 'Nepal', 'Nicaragua', 'North Vietnam', 'Panama', 'Paraguay', 'Portugal', 'Sierra Leone', 'Singapore', 'Solomon Islands', 'Somalia', 'South Vietnam', 'Sri Lanka', 'Swaziland', 'Togo', 'Uganda', 'Uruguay', 'Yemen, Dem. Rep. (South Yemen)', 'Zambia', 'Zimbabwe']
no_oil=['Afghanistan', 'Armenia', 'Bahamas, The', 'Belgium', 'Bhutan', 'Bosnia and Herzegovina', 'Botswana', 'Burkina Faso', 'Burundi', 'Cambodia', 'Cape Verde', 'Central African Republic', 'Comoros', 'Costa Rica', 'Cyprus', 'Djibouti', 'Dominican Republic', 'El Salvador', 'Eritrea', 'Ethiopia', 'Ethiopia including Eritrea', 'Fiji', 'Finland', 'Gambia, The', 'Guinea', 'Guinea-Bissau', 'Guyana', 'Haiti', 'Honduras', 'Iceland', 'Ireland', 'Jamaica', 'Kenya', 'Korea, Dem. Rep.', 'Korea, Rep.', 'Lao PDR', 'Latvia', 'Lebanon', 'Lesotho', 'Liberia', 'Luxembourg', 'Macedonia, FYR', 'Madagascar', 'Malawi', 'Maldives', 'Mali', 'Malta', 'Mauritius', 'Moldova', 'Montenegro', 'Mozambique', 'Namibia', 'Nepal', 'Nicaragua', 'North Vietnam', 'Panama', 'Paraguay', 'Portugal', 'Rwanda', 'Sierra Leone', 'Singapore', 'Solomon Islands', 'Somalia', 'South Vietnam', 'Sri Lanka', 'Swaziland', 'Switzerland', 'Tanzania', 'Togo', 'Uganda', 'Uruguay', 'Yemen, Dem. Rep. (South Yemen)', 'Zambia', 'Zimbabwe']
no_gas=['Armenia', 'Bahamas, The', 'Belize', 'Benin', 'Bhutan', 'Bosnia and Herzegovina', 'Botswana', 'Burkina Faso', 'Burundi', 'Cambodia', 'Cape Verde', 'Central African Republic', 'Comoros', 'Costa Rica', 'Cyprus', 'Djibouti', 'Dominican Republic', 'East Timor', 'El Salvador', 'Eritrea', 'Estonia', 'Ethiopia', 'Ethiopia including Eritrea', 'Fiji', 'Gambia, The', 'Guinea', 'Guinea-Bissau', 'Guyana', 'Haiti', 'Honduras', 'Iceland', 'Jamaica', 'Kenya', 'Korea, Dem. Rep.', 'Lao PDR', 'Latvia', 'Lebanon', 'Lesotho', 'Liberia', 'Lithuania', 'Macedonia, FYR', 'Madagascar', 'Malawi', 'Maldives', 'Mali', 'Malta', 'Mauritania', 'Mauritius', 'Mongolia', 'Montenegro', 'Namibia', 'Nepal', 'Nicaragua', 'Niger', 'North Vietnam', 'Panama', 'Paraguay', 'Portugal', 'Sierra Leone', 'Singapore', 'Solomon Islands', 'Somalia', 'South Sudan', 'South Vietnam', 'Sri Lanka', 'Sudan', 'Sudan including South Sudan', 'Suriname', 'Swaziland', 'Sweden', 'Togo', 'Uganda', 'Uruguay', 'Yemen Arab Rep. (North Yemen)', 'Yemen, Dem. Rep. (South Yemen)', 'Zambia', 'Zimbabwe']

def Lmodel(city,Target):
    if city not in no_oil:
        colr = Final_oil.split("~")[1].split("+");colr.append('oil_value_2014')
    else:
        colr = Final_gas.split("~")[1].split("+");colr.append('gas_value_2014')
    try:    
        X = pd.DataFrame.copy(df.loc[df.cty_name == city,colr]).reset_index()
        X[Target].replace(0,np.nan,inplace=True);X.dropna(inplace=True)
        _ = X.pop('index')
        Xtrain = X.iloc[0:round(X.shape[0]*0.7),:]
        Xtest = X.iloc[round(X.shape[0]*0.7):,:]
        Ytrain = Xtrain.pop(Target).reset_index()
        _ = Ytrain.pop('index')
        Ytest = Xtest.pop(Target).reset_index()
        _ = Ytest.pop('index')
        Tryear =Xtrain[['year']].reset_index()
        _ = Tryear.pop('index')
        Teyear = Xtest[['year']].reset_index()
        _ = Teyear.pop('index')
        
        lrm = LinearRegression()
        lrm.fit(Xtrain,Ytrain)
        
        fig = plt.figure(figsize=(18,6))
        ax1 = fig.add_subplot(1,2,1)
        ax2 = fig.add_subplot(1,2,2)
        pred = lrm.predict(Xtest)
        predd = pd.DataFrame(pred)
        lol = pd.concat([predd,Ytest,Teyear],axis=1)
        lol.columns = ['predicted','actual','year']
    
        pred2 = lrm.predict(Xtrain)
        predd2 = pd.DataFrame(pred2)
        lol2 = pd.concat([predd2,Ytrain,Tryear],axis=1)
        lol2.columns = ['predicted','actual','year']

        print(lol2[['predicted','actual']].corr())
        print(f"RMSE is {np.mean((lol2.actual-lol2.predicted)**2)}")
        print(f"MAPE is {np.mean(abs((lol2.actual-lol2.predicted))/lol2.actual)}")
        print(lol[['predicted','actual']].corr())
        print(f"RMSE is {np.sqrt(np.mean((lol.actual-lol.predicted)**2))}")
        print(f"MAPE is {np.mean(abs((lol.actual-lol.predicted))/lol.actual)}")

        lol.plot(x = 'year',y = 'actual',ax=ax2)
        lol.plot(x = 'year',y = 'predicted',ax=ax2)
        ax2.set_title('Test')
        lol2.plot(x = 'year',y = 'actual',ax=ax1)
        lol2.plot(x = 'year',y = 'predicted',ax=ax1)
        ax1.set_title('Train')
    except:
        if city in no_oil and (Target=='oil_value_2014'):
            print('___________________________Here Oil Production is Zero_______')
        if city in no_gas and (Target=='gas_value_2014'):
            print('___________________________Here Gas Production is Zero_______')
interact(Lmodel,city = [i for i in sorted(list(set(df.cty_name)))if i not in no],Target=['oil_value_2014','gas_value_2014'])

interactive(children=(Dropdown(description='city', options=('Afghanistan', 'Albania', 'Algeria', 'Angola', 'Ar…

<function __main__.Lmodel(city, Target)>

In [15]:
#df[df['cty_name']=='Azerbaijan'][['year','oil_prod32_14']].tail(35)

# Wrraper Method Feature Selection

## Forward Selction

In [16]:
#  data=df.iloc[:,3:40];data.drop(['sovereign','eiacty'],axis=1,inplace=True)

In [17]:
#  x_train,x_test,y_train,y_test=train_test_split(data.drop('oil_value_2014',axis=1),data.oil_value_2014,test_size=0.25,random_state=1)

In [18]:
#  x_train.fillna(0,inplace=True);y_train.fillna(0,inplace=True)

In [19]:
# # !pip install mlxtend
# from mlxtend.feature_selection import SequentialFeatureSelector as sfs
# from sklearn.ensemble import RandomForestRegressor
# np.random.seed(42)
# model=sfs(RandomForestRegressor(),k_features=15,forward=True,verbose=2,cv=5,n_jobs=-1,scoring='r2')
# model.fit(x_train,y_train)

In [20]:
# print(list(model.k_feature_names_))

## Backward Elimination

In [21]:
# backward_Model=sfs(RandomForestRegressor(),k_features=10,forward=False,verbose=2,cv=5,n_jobs=-1,scoring='r2')
# np.random.seed(42)
# backward_Model.fit(np.array(x_train),y_train)

In [22]:
# backward_Model.k_feature_idx_

In [23]:
# c=x_train.columns[list(backward_Model.k_feature_idx_)]
# c

## Exhaustive Selection

In [24]:
# from mlxtend.feature_selection import ExhaustiveFeatureSelector as efs
# Exhaustive_model=efs(RandomForestRegressor(),min_features=3,max_features=5,scoring='r2',n_jobs=-1)
# np.random.seed(42)
# miniData=x_train[x_train.columns[list(backward_Model.k_feature_idx_)]]
# Exhaustive_model.fit(np.array(miniData),y_train)

In [25]:
# c[list(Exhaustive_model.best_idx_)]
